In [ ]:
# %load_ext autoreload
# %autoreload 2
import sys
sys.path.insert(0,'/home/michele/lavoro/code/librascal/build/')
#sys.path.insert(0,'/home/nigam/git/librascal_cs/librascal/build/')

This notebook provides examples of the kind of manipulations that need to be applied to rotate structures, spherical expansion coefficients and higher-order equivariants, which are useful to test equivariance of features and kernels, and in general to manipulate invariant and equivariant properties. 

# Spherical expansion coefficients

Demonstration of manipulations of spherical expansion coefficients, and of conversion between real-valued and complex-valued spherical harmonics

In [ ]:
from ase.io import read
import ase
from copy import deepcopy
import numpy as np
import matplotlib.pyplot as plt

from rascal.representations import SphericalExpansion, SphericalInvariants
from rascal.utils import (get_radial_basis_covariance, get_radial_basis_pca, 
                          get_radial_basis_projections, get_optimal_radial_basis_hypers )
from rascal.utils import radial_basis
from rascal.utils import (WignerDReal, ClebschGordanReal, 
                          spherical_expansion_reshape, spherical_expansion_conjugate,
                    lm_slice, real2complex_matrix)

In [ ]:
# imports also some internals to demonstrate manually some CG manipulations
from rascal.utils.cg_utils import _r2c as r2c
from rascal.utils.cg_utils import _c2r as c2r
from rascal.utils.cg_utils import _cg as clebsch_gordan
from rascal.utils.cg_utils import _rotation as rotation
from rascal.utils.cg_utils import _wigner_d as wigner_d

## Loads the structures

In [ ]:
import urllib.request
# a collection of distorted ethanol molecules from the ANI-1 dataset 
# (see https://github.com/isayev/ANI1_dataset) with energies and forces computed using DFTB+ 
# (see https://www.dftbplus.org/)
url = 'https://raw.githubusercontent.com/cosmo-epfl/librascal-example-data/833b4336a7daf471e16993158322b3ea807b9d3f/inputs/molecule_conformers_dftb.xyz'
# Download the file from `url`, save it in a temporary directory and get the
# path to it (e.g. '/tmp/tmpb48zma.txt') in the `structures_fn` variable:
structures_fn, headers = urllib.request.urlretrieve(url)
structures_fn

In [ ]:
# Total number of structure to load
N = 100

# load the structures
frames = read(structures_fn,':{}'.format(N))

## Demonstrate the equivariance of spherical expansion coefficients

first, we compute the density expansion coefficients on a representative dataset

In [ ]:
spherical_expansion_hypers = {
    "interaction_cutoff": 3,
    "max_radial": 8,
    "max_angular": 6,
    "gaussian_sigma_constant": 0.3,
    "gaussian_sigma_type": "Constant",
    "cutoff_smooth_width": 0.5,
    "radial_basis": "GTO",
}

spex = SphericalExpansion(**spherical_expansion_hypers)

In [ ]:
selframe = frames[8];  sel_l = 3;    # frame and l value used for the test
feat_scaling = 1e6                   # just a scaling to make coefficients O(1)
feats = spex.transform(selframe).get_features(spex)
ref_feats = feat_scaling*spherical_expansion_reshape(feats, **spherical_expansion_hypers)

In [ ]:
# random rotation in terms of Euler angles
abc = np.random.uniform(size=(3))*np.pi

In [ ]:
# this is the Cartesian rotation matrix (helper function, follows ZYZ convention)
mrot = rotation(*abc)

In [ ]:
# computes the rotated structure and the associated features
rotframe = selframe.copy()
rotframe.positions = rotframe.positions @ mrot.T
rotframe.cell = rotframe.cell @ mrot.T   # rotate also the cell
rotfeats = spex.transform(rotframe).get_features(spex)
ref_feats_rot = feat_scaling*spherical_expansion_reshape(rotfeats, **spherical_expansion_hypers)

In [ ]:
print(ref_feats[0,0,0,lm_slice(sel_l)])
np.linalg.norm(ref_feats[0,0,0,lm_slice(sel_l)])

the coefficients have the same magnitude, but they differ because of rotation!

In [ ]:
print(ref_feats_rot[0,0,0,lm_slice(sel_l)])
np.linalg.norm(ref_feats_rot[0,0,0,lm_slice(sel_l)])

## Rotate the spherical expansion features using Wigner matrices

In [ ]:
# computing the wigner matrix takes some time for L>4
rotation_d = wigner_d(sel_l, *abc)

In [ ]:
# D^l_mm is orthogonal
plt.matshow(np.real(np.conjugate(rotation_d.T)@rotation_d))

In [ ]:
#  back and forth to check transformation from real to complex SPH
c2r(r2c(ref_feats[0,0,0,lm_slice(sel_l)])) - ref_feats[0,0,0,lm_slice(sel_l)]

Rotating the spherical harmonics using the Wigner D matrix formula
$\langle nlm|\hat{R} A; \rho_i \rangle = 
\sum_{mm'} D^l_{mm'}(\hat{R})^\star \langle nlm'|A; \rho_i \rangle
$

In [ ]:
ref_feats[0,0,0,lm_slice(sel_l)]

In [ ]:
ref_feats_rot[0,0,0,lm_slice(sel_l)]

In [ ]:
c2r(np.conjugate(rotation_d)@r2c(ref_feats[0,0,0,lm_slice(sel_l)]))

## Direct real transformations

There's no "real" reason to go through the complex algebra for rotations - we can transform once and for all the coefficients and be done with that!

Key idea is that the complex<->real transformations can be formulated in a matrix form

In [ ]:
# matrix version of the real-2-complex and complex-2-real transformations
r2c_mat = real2complex_matrix(sel_l)
c2r_mat = np.conjugate(r2c_mat.T)

.... which can be used to transform features between complex and real

In [ ]:
# we can use this to transform features
r2c_mat@ref_feats[0,0,0,lm_slice(sel_l)] - r2c(ref_feats[0,0,0,lm_slice(sel_l)])

... but can also be applied to matrices that act on the features

In [ ]:
# and Wigner D matrix as well
real_rotation_d = np.real(c2r_mat @ np.conjugate(rotation_d) @ r2c_mat)

The direct real rotation is equal (modulo noise) to going back and forth from complex sph

In [ ]:
real_rotation_d @ ref_feats[0,0,0,lm_slice(sel_l)] - ref_feats_rot[0,0,0,lm_slice(sel_l)]

# Clebsch-Gordan iteration

CG coefficients can be used to combine covariant objects so that, in turn, they compute as covariant objects. This is the idea behind NICE [[original paper](doi.org/10.1063/5.0021116)], as well as covariant neural networks

In [ ]:
# these are the indices of the features 
sl1, sl2, sL = 3, 2, sel_l
cg = clebsch_gordan(sl1, sl2, sL)

In [ ]:
cg_feats = np.einsum("abc,a,b->c", cg,
                    r2c(ref_feats[0,0,0,lm_slice(sl1)]), 
                    r2c(ref_feats[0,0,0,lm_slice(sl2)]))

In [ ]:
cg_feats_rot = np.einsum("abc,a,b->c", cg,
                    r2c(ref_feats_rot[0,0,0,lm_slice(sl1)]), 
                    r2c(ref_feats_rot[0,0,0,lm_slice(sl2)]))

In [ ]:
cg_feats

In [ ]:
cg_feats_rot

In [ ]:
np.conjugate(rotation_d)@cg_feats

## Real form of the iteration

the CG iteration can also be cast in a way so it acts directly on the real-valued coefficients

In [ ]:
r2c_mat_l1 = real2complex_matrix(sl1)
r2c_mat_l2 = real2complex_matrix(sl2)
r2c_mat_L = real2complex_matrix(sL)

computing the real-valued CGs requires converting in the appropriate way inputs AND outputs

In [ ]:
real_cg = np.real(np.einsum("abc, ax, by, zc -> xyz", cg, r2c_mat_l1, r2c_mat_l2, np.conjugate(r2c_mat_L.T)))

while the "complex" CG have a simple sparsity pattern (m1+m2=M), the real-valued are kind of messy because they need to pick up and combine real and imaginary parts of the expansion coefficients

In [ ]:
cg[:,:,2]

In [ ]:
real_cg[:,:,2]

... but at the end of the day, they work just fine!

In [ ]:
real_cg_feats = np.einsum("abc,a,b->c",real_cg,
                    ref_feats[0,0,0,lm_slice(sl1)],
                    ref_feats[0,0,0,lm_slice(sl2)])

In [ ]:
real_cg_feats - c2r(cg_feats)

needless to say, these are also equivariant, and can be acted upon with the real-valued wigner matrix

In [ ]:
real_rotation_d @ real_cg_feats - c2r(cg_feats_rot)

# Streamlined WignerD, and CG class

Uses the utility classes defined in rascal.utils to do all of the above (and more!)
`WignerDReal` is a Wigner D matrix implementation to rotate Y^m_l - like coefficients, while
`ClebschGordanReal` precomputes Clebsch-Gordan operations using real-only storage of the spherical expansion coefficients. 
WignerDReal also allows rotating structures so you won't have to wonder about what Euler angle convention is being used ever again

In [ ]:
WD = WignerDReal(spherical_expansion_hypers["max_angular"], *abc)
CG = ClebschGordanReal(spherical_expansion_hypers["max_angular"])

computes a list of features for different l's, just to use for testing

In [ ]:
test_feats = [ ref_feats[0,0,0,lm_slice(l)]  for l in range(0,5) ]
test_feats_rot = [ ref_feats_rot[0,0,0,lm_slice(l)] for l in range(0,5) ]

## Rotation and CG iteration

frame rotation demo

In [ ]:
test_frame = WD.rotate_frame(selframe.copy())

In [ ]:
test_frame.positions - rotframe.positions

the CG iteration takes two equivariants, infers the l order by their size, and produces and invariant of the specified L order

In [ ]:
t1 = CG.combine(test_feats[3], test_feats[4], sel_l)
t1_r = CG.combine(test_feats_rot[3], test_feats_rot[4], sel_l)

In [ ]:
t1

this can be rotated with the WignerD helper (and, ça va sans dire, matches the equivariant computed for the rotated structure)

In [ ]:
WD.rotate(t1)

In [ ]:
t1_r

lots of fun: we can rinse, repeat as much as we want

In [ ]:
t2 = CG.combine(t1, test_feats[3], 2)
t2_r = CG.combine(t1_r, test_feats_rot[3], 2)

In [ ]:
WD.rotate(t2)

In [ ]:
t2_r

In [ ]:
t3 = CG.combine(t2, test_feats[3], 1)
t3_r = CG.combine(t2_r, test_feats_rot[3], 1)

In [ ]:
WD.rotate(t3)

In [ ]:
t3_r

In [ ]:
t4 = CG.combine(t3, t2, 3)
t4_r = CG.combine(t3_r, t2_r, 3)

In [ ]:
WD.rotate(t4)

In [ ]:
t4_r

note that the CG iter is built to fail gracefully if called with "impossible" inputs (e.g. with l1,l2,L that do not fulfill the triangle inequality)

In [ ]:
CG.combine(t2, t4, 6)  #nb: t2 is L=2 and t4 is L=3

... but not when called outside the precomputed range 

In [ ]:
CG.combine(t2, t4, 12)

## Feature products

Another common use of CG coefficients is to expand products of spherical harmonics into objects that transform as individual irreps of $O(3)$. This is also implemented as part of `ClebschGordanReal`. These are basically outer products of the features $|l_1 m1; l_2 m_2;\rangle = |l_1 m1\rangle |l_2 m_2;\rangle$

In [ ]:
test_prod = test_feats[2][:,np.newaxis]@test_feats[3][np.newaxis,:]
test_prod_rot = test_feats_rot[2][:,np.newaxis]@test_feats_rot[3][np.newaxis,:]

In [ ]:
plt.matshow(test_prod); plt.show(); plt.matshow(test_prod_rot);

`ClebschGordanReal.couple()` takes one of these $l_1\times l_2$ matrices and explodes them as a list of terms with $L\in[|l_1-l_2|,l_1+l_2|$.

In [ ]:
test_coupled = CG.couple(test_prod)

depending on the $l$ values this come from, the $L$ items have different nature (e.g. in terms of parity) and so the coupling function returns a tuple that also keeps track of the original shape of the product matrix

In [ ]:
test_coupled

the entries transform as $Y^m_l$ and can be rotated accordingly

In [ ]:
test_coupled_rot = CG.couple(test_prod_rot)

In [ ]:
test_coupled_rot[1][3]

In [ ]:
WD.rotate(test_coupled[1][3])

the coupled coefficients can be translated back into the product form

In [ ]:
test_decoupled = CG.decouple(test_coupled)

In [ ]:
np.linalg.norm(test_prod - test_decoupled)

this is a consequence of the fact that the real CG are orthogonal, just like their conventional counterparts

In [ ]:
# this also gives a view into the internal storage of the CG coefficients, that are stored in a
# sparse format because of the non-trivial sparsity pattern
l1,l2 = test_coupled[0]
prod = np.zeros((2*l1+1,2*l2+1,2*l1+1,2*l2+1))
for L in range(abs(l1-l2), abs(l1+l2)+1):
    for M in range(0, 2*L+1):
        for m1, m2, mcg in CG._cgdict[(l1, l2, L)][M]:
            for m1p, m2p, mcgp in CG._cgdict[(l1, l2, L)][M]:
                prod[m1,m2,m1p,m2p] += mcg*mcgp

In [ ]:
pr = prod.reshape((2*l1+1)*(2*l2+1),(2*l1+1)*(2*l2+1))
plt.matshow(pr)